<a href="https://colab.research.google.com/github/oliverferry/Neural-Style-Transfer-/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

batchSize = 64 
imageSize = 64 

transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.

dataset = dset.CIFAR10(root = './data', download = True, transform = transform) # We download the training set in the ./data folder and we apply the previous transformations on each image.
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # We use dataLoader to get the images of the training set batch by batch.

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input)
        return output

netG = G()
netG.apply(weights_init)


class D(nn.Module):

    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1)

netD = D()
netD.apply(weights_init)

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

for epoch in range(25):

    for i, data in enumerate(dataloader, 0):
        
        netD.zero_grad()
        
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output, target)
        
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())
        errD_fake = criterion(output, target)
        
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.data.item(), errG.data.item()))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/transforms.py:208: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Files already downloaded and verified
[0/25][0/782] Loss_D: 1.7400 Loss_G: 6.7989
[0/25][1/782] Loss_D: 1.1511 Loss_G: 4.5019
[0/25][2/782] Loss_D: 1.3344 Loss_G: 5.6742
[0/25][3/782] Loss_D: 0.7322 Loss_G: 6.4577
[0/25][4/782] Loss_D: 0.7921 Loss_G: 6.1079
[0/25][5/782] Loss_D: 0.9374 Loss_G: 6.9200
[0/25][6/782] Loss_D: 0.7352 Loss_G: 8.1701
[0/25][7/782] Loss_D: 0.6933 Loss_G: 8.4335
[0/25][8/782] Loss_D: 0.7575 Loss_G: 7.1458
[0/25][9/782] Loss_D: 1.2264 Loss_G: 10.4385
[0/25][10/782] Loss_D: 0.7060 Loss_G: 7.4607
[0/25][11/782] Loss_D: 1.1375 Loss_G: 11.3795
[0/25][12/782] Loss_D: 0.5228 Loss_G: 9.0312
[0/25][13/782] Loss_D: 0.5849 Loss_G: 8.5051
[0/25][14/782] Loss_D: 0.6682 Loss_G: 12.7232
[0/25][15/782] Loss_D: 0.5879 Loss_G: 9.3691
[0/25][16/782] Loss_D: 0.7547 Loss_G: 11.2860
[0/25][17/782] Loss_D: 0.3084 Loss_G: 9.0839
[0/25][18/782] Loss_D: 0.8825 Loss_G: 13.8194
[0/25][19/782] Loss_D: 0.1573 Loss_G: 10.9649
[0/25][20/782] Loss_D: 0.5758 Loss_G: 9.7738
[0/25][21/782] Loss_D

[0/25][178/782] Loss_D: 0.5740 Loss_G: 5.5281
[0/25][179/782] Loss_D: 0.1546 Loss_G: 3.3023
[0/25][180/782] Loss_D: 0.3924 Loss_G: 6.5661
[0/25][181/782] Loss_D: 0.4674 Loss_G: 4.4098
[0/25][182/782] Loss_D: 0.3082 Loss_G: 5.5566
[0/25][183/782] Loss_D: 0.2039 Loss_G: 4.8679
[0/25][184/782] Loss_D: 0.4783 Loss_G: 3.5209
[0/25][185/782] Loss_D: 1.0626 Loss_G: 8.2411
[0/25][186/782] Loss_D: 0.9538 Loss_G: 3.5827
[0/25][187/782] Loss_D: 0.6840 Loss_G: 5.9491
[0/25][188/782] Loss_D: 0.6836 Loss_G: 4.0015
[0/25][189/782] Loss_D: 0.5192 Loss_G: 5.4549
[0/25][190/782] Loss_D: 0.4255 Loss_G: 3.8063
[0/25][191/782] Loss_D: 0.3085 Loss_G: 4.5898
[0/25][192/782] Loss_D: 0.3715 Loss_G: 3.3433
[0/25][193/782] Loss_D: 0.6071 Loss_G: 6.4720
[0/25][194/782] Loss_D: 0.9617 Loss_G: 2.1042
[0/25][195/782] Loss_D: 0.9931 Loss_G: 6.7742
[0/25][196/782] Loss_D: 1.6916 Loss_G: 2.4334
[0/25][197/782] Loss_D: 0.7220 Loss_G: 5.0891
[0/25][198/782] Loss_D: 0.4783 Loss_G: 3.9620
[0/25][199/782] Loss_D: 0.3645 Los

[0/25][356/782] Loss_D: 0.1165 Loss_G: 3.8660
[0/25][357/782] Loss_D: 0.3865 Loss_G: 7.8616
[0/25][358/782] Loss_D: 0.1177 Loss_G: 7.8709
[0/25][359/782] Loss_D: 0.3432 Loss_G: 4.3839
[0/25][360/782] Loss_D: 0.4115 Loss_G: 8.7669
[0/25][361/782] Loss_D: 0.1377 Loss_G: 8.6491
[0/25][362/782] Loss_D: 0.2272 Loss_G: 4.7509
[0/25][363/782] Loss_D: 0.9267 Loss_G: 14.1600
[0/25][364/782] Loss_D: 1.0273 Loss_G: 12.4023
[0/25][365/782] Loss_D: 0.2037 Loss_G: 8.5732
[0/25][366/782] Loss_D: 0.0721 Loss_G: 5.5432
[0/25][367/782] Loss_D: 0.3342 Loss_G: 8.0342
[0/25][368/782] Loss_D: 0.1268 Loss_G: 7.7926
[0/25][369/782] Loss_D: 0.1134 Loss_G: 5.8128
[0/25][370/782] Loss_D: 0.3436 Loss_G: 4.5405
[0/25][371/782] Loss_D: 0.9147 Loss_G: 8.3123
[0/25][372/782] Loss_D: 0.1979 Loss_G: 8.0765
[0/25][373/782] Loss_D: 0.2274 Loss_G: 4.9097
[0/25][374/782] Loss_D: 0.4779 Loss_G: 9.6365
[0/25][375/782] Loss_D: 0.8438 Loss_G: 7.1271
[0/25][376/782] Loss_D: 0.1321 Loss_G: 4.7116
[0/25][377/782] Loss_D: 0.1885 L

[0/25][535/782] Loss_D: 0.0533 Loss_G: 5.6972
[0/25][536/782] Loss_D: 0.1101 Loss_G: 5.0628
[0/25][537/782] Loss_D: 0.2054 Loss_G: 3.2473
[0/25][538/782] Loss_D: 0.4422 Loss_G: 6.5981
[0/25][539/782] Loss_D: 0.5831 Loss_G: 3.7239
[0/25][540/782] Loss_D: 0.2662 Loss_G: 3.7795
[0/25][541/782] Loss_D: 0.2449 Loss_G: 5.5351
[0/25][542/782] Loss_D: 0.0764 Loss_G: 5.9106
[0/25][543/782] Loss_D: 0.1495 Loss_G: 4.8266
[0/25][544/782] Loss_D: 0.1012 Loss_G: 4.6195
[0/25][545/782] Loss_D: 0.4043 Loss_G: 3.3468
[0/25][546/782] Loss_D: 0.3673 Loss_G: 7.2233
[0/25][547/782] Loss_D: 1.3769 Loss_G: 1.1741
[0/25][548/782] Loss_D: 1.5577 Loss_G: 13.0125
[0/25][549/782] Loss_D: 0.5519 Loss_G: 11.4529
[0/25][550/782] Loss_D: 0.5488 Loss_G: 6.0280
[0/25][551/782] Loss_D: 0.5905 Loss_G: 5.4254
[0/25][552/782] Loss_D: 0.5230 Loss_G: 6.2644
[0/25][553/782] Loss_D: 0.5054 Loss_G: 3.7641
[0/25][554/782] Loss_D: 0.5405 Loss_G: 4.7770
[0/25][555/782] Loss_D: 0.4761 Loss_G: 4.2935
[0/25][556/782] Loss_D: 0.3344 L

...

[9/25][673/782] Loss_D: 0.3526 Loss_G: 3.4640
[9/25][674/782] Loss_D: 0.2380 Loss_G: 3.6826
[9/25][675/782] Loss_D: 0.2286 Loss_G: 3.1598
[9/25][676/782] Loss_D: 0.1144 Loss_G: 3.7933
[9/25][677/782] Loss_D: 0.2406 Loss_G: 4.2560
[9/25][678/782] Loss_D: 0.2294 Loss_G: 4.6938
[9/25][679/782] Loss_D: 0.1538 Loss_G: 4.0693
[9/25][680/782] Loss_D: 0.1484 Loss_G: 3.1701
[9/25][681/782] Loss_D: 0.0494 Loss_G: 3.5090
[9/25][682/782] Loss_D: 0.2330 Loss_G: 5.2477
[9/25][683/782] Loss_D: 0.0542 Loss_G: 5.7884
[9/25][684/782] Loss_D: 0.0935 Loss_G: 4.6857
[9/25][685/782] Loss_D: 0.0930 Loss_G: 6.2001
[9/25][686/782] Loss_D: 0.0213 Loss_G: 4.9541
[9/25][687/782] Loss_D: 0.0522 Loss_G: 4.0363
[9/25][688/782] Loss_D: 0.1834 Loss_G: 5.2584
[9/25][689/782] Loss_D: 0.1619 Loss_G: 4.3045
[9/25][690/782] Loss_D: 0.1636 Loss_G: 4.2923
[9/25][691/782] Loss_D: 0.2725 Loss_G: 3.5793
[9/25][692/782] Loss_D: 0.1184 Loss_G: 3.5767
[9/25][693/782] Loss_D: 0.1436 Loss_G: 4.0620
[9/25][694/782] Loss_D: 0.0853 Los

[10/25][70/782] Loss_D: 1.1508 Loss_G: 1.0271
[10/25][71/782] Loss_D: 1.2459 Loss_G: 4.1264
[10/25][72/782] Loss_D: 1.5568 Loss_G: 0.7701
[10/25][73/782] Loss_D: 1.3758 Loss_G: 4.4068
[10/25][74/782] Loss_D: 0.8690 Loss_G: 2.5129
[10/25][75/782] Loss_D: 0.5101 Loss_G: 2.1532
[10/25][76/782] Loss_D: 0.6979 Loss_G: 3.6998
[10/25][77/782] Loss_D: 0.5517 Loss_G: 2.7494
[10/25][78/782] Loss_D: 0.8757 Loss_G: 1.4410
[10/25][79/782] Loss_D: 0.7949 Loss_G: 2.9194
[10/25][80/782] Loss_D: 0.5469 Loss_G: 3.2930
[10/25][81/782] Loss_D: 0.8650 Loss_G: 1.5756
[10/25][82/782] Loss_D: 1.0550 Loss_G: 2.5868
[10/25][83/782] Loss_D: 0.6456 Loss_G: 2.5945
[10/25][84/782] Loss_D: 0.7534 Loss_G: 1.4112
[10/25][85/782] Loss_D: 0.8954 Loss_G: 3.1643
[10/25][86/782] Loss_D: 0.6965 Loss_G: 2.5631
[10/25][87/782] Loss_D: 0.4348 Loss_G: 2.3155
[10/25][88/782] Loss_D: 0.5121 Loss_G: 2.8208
[10/25][89/782] Loss_D: 0.5801 Loss_G: 2.5186
[10/25][90/782] Loss_D: 0.6562 Loss_G: 1.8925
[10/25][91/782] Loss_D: 0.4902 Los

[10/25][245/782] Loss_D: 0.5161 Loss_G: 2.8851
[10/25][246/782] Loss_D: 0.7651 Loss_G: 1.8234
[10/25][247/782] Loss_D: 0.6091 Loss_G: 2.9492
[10/25][248/782] Loss_D: 0.4295 Loss_G: 2.9902
[10/25][249/782] Loss_D: 0.4778 Loss_G: 2.2304
[10/25][250/782] Loss_D: 0.7522 Loss_G: 2.1638
[10/25][251/782] Loss_D: 0.4770 Loss_G: 3.5328
[10/25][252/782] Loss_D: 0.3997 Loss_G: 2.6819
[10/25][253/782] Loss_D: 0.4078 Loss_G: 2.4547
[10/25][254/782] Loss_D: 0.5758 Loss_G: 3.3811
[10/25][255/782] Loss_D: 0.4201 Loss_G: 2.7553
[10/25][256/782] Loss_D: 0.6468 Loss_G: 1.4719
[10/25][257/782] Loss_D: 1.1128 Loss_G: 1.9334
[10/25][258/782] Loss_D: 0.5395 Loss_G: 3.0635
[10/25][259/782] Loss_D: 0.4335 Loss_G: 3.1485
[10/25][260/782] Loss_D: 0.4652 Loss_G: 2.9783
[10/25][261/782] Loss_D: 0.6947 Loss_G: 1.3855
[10/25][262/782] Loss_D: 0.8138 Loss_G: 3.7429
[10/25][263/782] Loss_D: 0.4773 Loss_G: 3.0166
[10/25][264/782] Loss_D: 0.4953 Loss_G: 2.0232
[10/25][265/782] Loss_D: 0.5622 Loss_G: 3.4802
[10/25][266/7

[10/25][420/782] Loss_D: 0.3722 Loss_G: 2.8607
[10/25][421/782] Loss_D: 0.4497 Loss_G: 1.7760
[10/25][422/782] Loss_D: 0.6335 Loss_G: 4.2751
[10/25][423/782] Loss_D: 0.7109 Loss_G: 1.2582
[10/25][424/782] Loss_D: 0.5532 Loss_G: 4.8635
[10/25][425/782] Loss_D: 0.2603 Loss_G: 4.1608
[10/25][426/782] Loss_D: 0.1270 Loss_G: 3.4538
[10/25][427/782] Loss_D: 0.1322 Loss_G: 3.6263
[10/25][428/782] Loss_D: 0.1079 Loss_G: 3.9757
[10/25][429/782] Loss_D: 0.1386 Loss_G: 4.0922
[10/25][430/782] Loss_D: 0.0900 Loss_G: 3.6690
[10/25][431/782] Loss_D: 0.0799 Loss_G: 3.9666
[10/25][432/782] Loss_D: 0.1672 Loss_G: 4.4161
[10/25][433/782] Loss_D: 0.0754 Loss_G: 4.8500
[10/25][434/782] Loss_D: 0.1341 Loss_G: 3.3033
[10/25][435/782] Loss_D: 0.1123 Loss_G: 3.7216
[10/25][436/782] Loss_D: 0.0929 Loss_G: 3.7865
[10/25][437/782] Loss_D: 0.1682 Loss_G: 3.2628
[10/25][438/782] Loss_D: 0.1310 Loss_G: 4.0706
[10/25][439/782] Loss_D: 0.0776 Loss_G: 4.7629
[10/25][440/782] Loss_D: 0.0944 Loss_G: 5.0685
[10/25][441/7

[10/25][595/782] Loss_D: 0.0091 Loss_G: 7.0884
[10/25][596/782] Loss_D: 0.0394 Loss_G: 4.7199
[10/25][597/782] Loss_D: 0.0138 Loss_G: 7.9316
[10/25][598/782] Loss_D: 0.0048 Loss_G: 7.2581
[10/25][599/782] Loss_D: 0.0205 Loss_G: 5.0566
[10/25][600/782] Loss_D: 0.0199 Loss_G: 5.2664
[10/25][601/782] Loss_D: 0.0095 Loss_G: 6.5343
[10/25][602/782] Loss_D: 0.0197 Loss_G: 5.2257
[10/25][603/782] Loss_D: 0.0231 Loss_G: 5.5964
[10/25][604/782] Loss_D: 0.0310 Loss_G: 5.0193
[10/25][605/782] Loss_D: 0.0115 Loss_G: 5.5062
[10/25][606/782] Loss_D: 0.0280 Loss_G: 4.8871
[10/25][607/782] Loss_D: 0.0074 Loss_G: 6.9372
[10/25][608/782] Loss_D: 0.4004 Loss_G: 10.6954
[10/25][609/782] Loss_D: 3.6795 Loss_G: 6.6678
[10/25][610/782] Loss_D: 4.2424 Loss_G: 0.0190
[10/25][611/782] Loss_D: 4.9544 Loss_G: 2.5014
[10/25][612/782] Loss_D: 0.6804 Loss_G: 6.5637
[10/25][613/782] Loss_D: 1.4700 Loss_G: 2.8414
[10/25][614/782] Loss_D: 0.6325 Loss_G: 2.6517
[10/25][615/782] Loss_D: 0.3560 Loss_G: 3.9326
[10/25][616/